In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score

In [2]:
class artificial_neuron:
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

    def initialisation(self, n0, n1, n2):
        w1 = np.random.randn(n1, n0)
        b1 = np.random.randn(n1, 1)
        w2 = np.random.randn(n2, n1)
        b2 = np.random.randn(n2, 1)

        parameters = {
            'w1': w1,
            'b1': b1,
            'w2': w2,
            'b2': b2
        }
        return parameters

    def model(self, X, parameters):
        w1 = parameters['w1']
        b1 = parameters['b1']
        w2 = parameters['w2']
        b2 = parameters['b2']

        # Forward propagation
        z1 = w1.dot(X.T) + b1
        A1 = 1 / (1 + np.exp(-z1))
        z2 = w2.dot(A1) + b2
        A2 = 1 / (1 + np.exp(-z2))

        activations = {
            'A1': A1,
            'A2': A2,
        }

        return activations

    def log_loss(self, A, y):
        ep = 1e-15
        A = np.clip(A, ep, 1 - ep)  # Avoid log(0)
        return np.mean(-y * np.log(A) - (1 - y) * np.log(1 - A))

    def back_propagation(self, X, y, activations, parameters):
        A1 = activations['A1']
        A2 = activations['A2']
        w2 = parameters['w2']
        m = y.shape[0]

        # check if y is properly shaped for matrix operations
        y_reshaped = y.reshape(1, -1)

        # Backpropagation for output layer
        dz2 = A2 - y_reshaped
        dw2 = (1/m) * dz2.dot(A1.T)
        db2 = (1/m) * np.sum(dz2, axis=1, keepdims=True)

        # Backpropagation for hidden layer
        dz1 = w2.T.dot(dz2) * A1 * (1 - A1)
        dw1 = (1/m) * dz1.dot(X)
        db1 = (1/m) * np.sum(dz1, axis=1, keepdims=True)

        gradients = {
            'dw1': dw1,
            'db1': db1,
            'dw2': dw2,
            'db2': db2,
        }
        return gradients

    def update(self, parameters, gradients, learning_rate):
        w1 = parameters['w1']
        b1 = parameters['b1']
        w2 = parameters['w2']
        b2 = parameters['b2']

        dw1 = gradients['dw1']
        db1 = gradients['db1']
        dw2 = gradients['dw2']
        db2 = gradients['db2']

        w1 = w1 - learning_rate * dw1
        b1 = b1 - learning_rate * db1
        w2 = w2 - learning_rate * dw2
        b2 = b2 - learning_rate * db2

        parameters = {
            'w1': w1,
            'b1': b1,
            'w2': w2,
            'b2': b2
        }
        return parameters

    def predict(self, X, parameters):
        activations = self.model(X, parameters)
        A2 = activations['A2']
        return (A2 >= 0.5).astype(int)

    def ARN(self, n1=120, learning_rate=0.01, n_iter=1000):

        # Get dimensions
        n0 = self.X_train.shape[1]
        n2 = 1

        # Initialize parameters
        parameters = self.initialisation(n0, n1, n2)

        train_loss = []
        train_acc = []
        test_loss = []
        test_acc = []

        # Training loop
        for i in tqdm(range(n_iter)):

            # Forward propagation on training data
            activations_train = self.model(self.X_train, parameters)
            A2_train = activations_train['A2']

            # Calculate metrics every 100 iterations
            if i % 100 == 0:
                # Train
                train_loss.append(self.log_loss(A2_train.T, self.y_train))
                y_pred_train = self.predict(self.X_train, parameters)
                train_acc.append(accuracy_score(self.y_train, y_pred_train.T))

                # Test
                activations_test = self.model(self.X_test, parameters)
                A2_test = activations_test['A2']
                test_loss.append(self.log_loss(A2_test.T, self.y_test))
                y_pred_test = self.predict(self.X_test, parameters)
                test_acc.append(accuracy_score(self.y_test, y_pred_test.T))

            # Backward propagation and update
            gradients = self.back_propagation(self.X_train, self.y_train, activations_train, parameters)
            parameters = self.update(parameters, gradients, learning_rate)

        # Plot results
        plt.figure(figsize=(14, 6))

        plt.subplot(1, 2, 1)
        plt.plot(train_loss, label='Train Loss')
        plt.plot(test_loss, label='Test Loss')
        plt.xlabel('Iterations (x100)')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('Training and Test Loss')

        plt.subplot(1, 2, 2)
        plt.plot(train_acc, label='Train Accuracy')
        plt.plot(test_acc, label='Test Accuracy')
        plt.xlabel('Iterations (x100)')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.title('Training and Test Accuracy')

        plt.tight_layout()
        plt.show()


        return parameters

    def get_weights(self, parameters):
        return parameters['w1'], parameters['w2']

    def get_biases(self, parameters):
        return parameters['b1'], parameters['b2']